# Oracle LiveLabs - Forecasting service

[Oracle LiveLabs - Forecasting Service](https://apexapps.oracle.com/pls/apex/dbpm/r/livelabs/workshop-attendee-2?p210_workshop_id=927&p210_type=2&session=107363063846819)

In [1]:
import pandas as pd
import requests
import json
import ast
import matplotlib.pyplot as plt
import re
import os
import simplejson

import oci

# you can also use resource principal.
signer = oci.auth.signers.InstancePrincipalsSecurityTokenSigner()
signer

## download sample data

```bash
# primary data
wget https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-primary.csv

# additional data
wget https://oracle.github.io/learning-library/oci-library/oci-hol/oci-artificial-intelligence/forecasting-service/item-4-forecasting-understand-data/files/favorita-13-beverages-add.csv
```

In [29]:
# inline data preparation

# load data
df_primary = pd.read_csv('sample-data/favorita-13-beverages-primary.csv')
df_additional = pd.read_csv('sample-data/favorita-13-beverages-add.csv')

# modify data format to match forecasting service
df_primary['date'] = pd.to_datetime(df_primary['date'], format='%d/%m/%y').apply(lambda date: str(date))
df_additional['date'] = pd.to_datetime(df_additional['date'], format='%d/%m/%y').apply(lambda date: str(date))

# setting variables to create forecasting service
col_order_primary = ['date', 'sales', 'item_id']
col_order_additional = ['date', 'onpromotion', 'item_id']

primary_load = df_primary[col_order_primary].values.transpose().tolist()
additional_load = df_additional[col_order_additional].values.transpose().tolist()

primary_load
additional_load

[['2013-01-01 00:00:00',
  '2013-01-02 00:00:00',
  '2013-01-03 00:00:00',
  '2013-01-04 00:00:00',
  '2013-01-05 00:00:00',
  '2013-01-06 00:00:00',
  '2013-01-07 00:00:00',
  '2013-01-08 00:00:00',
  '2013-01-09 00:00:00',
  '2013-01-10 00:00:00',
  '2013-01-11 00:00:00',
  '2013-01-12 00:00:00',
  '2013-01-13 00:00:00',
  '2013-01-14 00:00:00',
  '2013-01-15 00:00:00',
  '2013-01-16 00:00:00',
  '2013-01-17 00:00:00',
  '2013-01-18 00:00:00',
  '2013-01-19 00:00:00',
  '2013-01-20 00:00:00',
  '2013-01-21 00:00:00',
  '2013-01-22 00:00:00',
  '2013-01-23 00:00:00',
  '2013-01-24 00:00:00',
  '2013-01-25 00:00:00',
  '2013-01-26 00:00:00',
  '2013-01-27 00:00:00',
  '2013-01-28 00:00:00',
  '2013-01-29 00:00:00',
  '2013-01-30 00:00:00',
  '2013-01-31 00:00:00',
  '2013-02-01 00:00:00',
  '2013-02-02 00:00:00',
  '2013-02-03 00:00:00',
  '2013-02-04 00:00:00',
  '2013-02-05 00:00:00',
  '2013-02-06 00:00:00',
  '2013-02-07 00:00:00',
  '2013-02-08 00:00:00',
  '2013-02-09 00:00:00',


In [3]:


url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/projects"

payload = json.dumps({
  "displayName": "Forecast API Demo",
  "compartmentId": os.environ['C'],
  "description": "Forecasting service API Demo",
  "freeformTags": None,
  "definedTags": None,
  "systemTags": None
})
headers = {
  'Content-Type': 'application/json'
}
response = requests.request("POST", url, headers=headers, data=payload, auth=signer)

In [ ]:
create_project_response = json.loads(response.text)
create_project_response

In [5]:
date_col_primary = 'date'
date_col_add = 'date'
target_col = 'sales'
id_col_primary = 'item_id'
id_col_add = 'item_id'
data_frequency = 'DAY'
forecast_frequency = 'DAY'
forecast_horizon = 14

In [6]:
%%time

forecast_url = "https://forecasting.aiservice.us-phoenix-1.oci.oraclecloud.com/20220101/forecasts"

forecast_payload = json.dumps({
    "compartmentId": os.environ['C'],
    "displayName": "Forecast Model",
    "description": "Training Forecast Model",
    "projectId": create_project_response.get('id'),
    "forecasCreationDetails": {
        "targetVariables": [
            target_col
        ]
    },
    "modelTrainingDetails": {
      "modelType": "UNIVARIATE",
      "models": [
        "SMA",
        "DMA",
        "HWSA",
        "HWSM",
        "SES",
        "DES",
        "SA",
        "SM",
        "UAM",
        "UHM",
        "ARIMA",
        "PROPHET"
      ]
    },
    "forecastHorizon": forecast_horizon,
    "confidenceInterval": "CI_5_95",
    "errorMeasure": "RMSE",
    "forecastTechnique": "ROCV",
    "forecastFrequency": forecast_frequency,
    "isForecastExplanationRequired": True,
    "dataSourceDetails": {
      "type": "INLINE",
      "dataSources": {
        "primaryDataSource": {
          "isDataGrouped": True,
          "columnData": primary_load,
          "columnSchema": [
            {
              "columnName": date_col_primary,
              "dataType": "DATE"
            },
            {
              "columnName": target_col,
              "dataType": "INT"
            },
            {
              "columnName": id_col_primary,
              "dataType": "STRING"
            },
          ],
          "tsColName": date_col_primary,
          "tsColFormat": "yyyy-MM-dd HH:mm:ss",
          "dataFrequency": data_frequency
        },
        "additionalDataSource": {
          "isDataGrouped": True,
          "columnData": additional_load,
          "columnSchema": [
            {
              "columnName": date_col_add,
              "dataType": "DATE"
            },
            {
              "columnName": "onpromotion",
              "dataType": "INT"
            },
            {
              "columnName": id_col_add,
              "dataType": "STRING"
            },
          ],
          "tsColName": date_col_add,
          "tsColFormat": "yyyy-MM-dd HH:mm:ss",
          "dataFrequency": data_frequency
        }
      }
    }
})

# print(forecast_payload)

forecast_headers = {
  "Content-Type": "application/json"
}

forecast_response = requests.request("POST", forecast_url, headers=forecast_headers, data=forecast_payload, auth=signer)

CPU times: user 18.4 ms, sys: 313 µs, total: 18.7 ms
Wall time: 925 ms


In [7]:
create_forecast_response = json.loads(forecast_response.text)
create_forecast_response

{'code': 'InternalError', 'message': 'Internal error occurred'}